# Setup an account

Get the tokens under:  
https://help.getharvest.com/api-v2/authentication-api/authentication/authentication/


## From manuals  
https://help.getharvest.com/api-v2/projects-api/projects/task-assignments/  

**Delete a task assignment**  
Delete a task assignment. Deleting a task assignment is only possible if it has no time entries associated with it. Returns a 200 OK response code if the call succeeded."  

**Update a task assignment**  
Updates the specific task assignment by setting the values of the parameters passed. Any parameters not provided will be left unchanged. Returns a task assignment object and a 200 OK response code if the call succeeded."  

**is_active** - boolean - Whether the task assignment is active or archived."

Da https://help.getharvest.com/api-v2/projects-api/projects/projects/

**Delete a project**  
Deletes a project and any time entries or expenses tracked to it. However, invoices associated with the project will not be deleted. __If you don’t want the project’s time entries and expenses to be deleted, you should archive the project instead.__"


# Libraries & Functions

***Important: run all cells until `2.1 Building DataFrames` included ***

In [1]:
import os
import json
import pandas as pd
import requests

In [2]:
def retrievePage(whichData='projects',page=1,per_page=100):
    '''
    Retrieves the url with the headers dict using the authorization from
    https://help.getharvest.com/api-v2/authentication-api/authentication/authentication/
    
    It takes:
    whichData: it can be 'projects', 'tasks' or 'clients'
    page: which page of the result is returned, if the number of entries is larger than
        per_page
    per_page: amount of entries per page, 1 <= per_page <= 100
    
    It returns:
    a json object containing the retrieved data 
    
    Example:
    retrievePage('projects', page=2) # it retrieves the 2nd page of the project list
    '''
    url = 'https://api.harvestapp.com/v2/' + whichData

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    payload = {'per_page':str(per_page),'page':str(page)}
    
    r = requests.get(url=url, headers=headers, params=payload)
    jsonResponse = r.json()
    return jsonResponse

In [3]:
def createJsonDict_Projects(whichData='projects'):
    '''
    It loops over the received pages in json form, it merges the results and it
    returns them. 
    It has to be used in combination with retrievePage(). 
    
    It takes:
    whichData: this function was specialized to work with projects but it can easily
    generalized to deal also with 'task' or with 'clients' (you have mainly to adapt
    which fields should be returned, in the inner loop)
        
    It returns:
    an array of dicts with fields: ProjName, ClientName, ClientID, ProjID
    '''
    
    # the output variable to the json array of ProjName, ClientName, ClientID, ProjID
    csvOut=[]
    # an empty call to retirieve page to infer how many entries and pages are available,
    # in order to setup the variables totalPages, totalEntries, perPage
    toExtract = retrievePage(whichData, page=1)
    totalPages = toExtract['total_pages']
    totalEntries = toExtract['total_entries']
    perPage = toExtract['per_page']
    
    #outer for loop on the pages (by default are 100 entries per page)
    for pg in range(1,totalPages+1):
        jsonOut = retrievePage(whichData, page=pg)
        # some calculations to know how many entries are in the page
        # to be used in the inner loop
        nrEntriesLeftOver = totalEntries - (pg-1)*perPage
        if nrEntriesLeftOver > 100:
            nrEntriesInLoop = 100
        else:
            nrEntriesInLoop = nrEntriesLeftOver
        #print(pg,nrEntriesInLoop)
        # loop over the entries in the page
        for entry in range(nrEntriesInLoop):
            temp = {}
            temp['ProjName'] = jsonOut[whichData][entry]['name']
            temp['ClientName'] = jsonOut[whichData][entry]['client']['name']
            temp['ClientID'] = jsonOut[whichData][entry]['client']['id']
            temp['ProjID'] = jsonOut[whichData][entry]['id']
            csvOut.append(temp)
    return csvOut

In [4]:
def createJsonDict_Clients(whichData='clients'):
    '''
    It loops over the received pages in json form, it merges the results and it
    returns them. 
    It has to be used in combination with retrievePage(). 
    
    It takes:
    whichData: this function was specialized to work with clients but it can easily
    generalized to deal also with 'task' or with 'projects' (you have mainly to adapt
    which fields should be returned, in the inner loop)
        
    It returns:
    an array of dicts with fields: ClientName, ClientID
    '''
    
    csvOut=[]
    toExtract = retrievePage(whichData, page=1)
    totalPages = toExtract['total_pages']
    totalEntries = toExtract['total_entries']
    perPage = toExtract['per_page']
    
    for pg in range(1,totalPages+1):
        jsonOut = retrievePage(whichData, page=pg)
        nrEntriesLeftOver = totalEntries - (pg-1)*perPage
        if nrEntriesLeftOver > 100:
            nrEntriesInLoop = 100
        else:
            nrEntriesInLoop = nrEntriesLeftOver
        #print(pg,nrEntriesInLoop)
        for entry in range(nrEntriesInLoop):
            temp = {}
            temp['ClientName'] = jsonOut[whichData][entry]['name']
            temp['ClientID'] = jsonOut[whichData][entry]['id']
            csvOut.append(temp)
    return csvOut

In [5]:
def createJsonDict_Tasks(whichData='tasks'):
    '''
    It loops over the received pages in json form, it merges the results and it
    returns them. 
    It has to be used in combination with retrievePage(). 
    
    It takes:
    whichData: this function was specialized to work with tasks but it can easily
    generalized to deal also with 'clients' or with 'projects' (you have mainly to adapt
    which fields should be returned, in the inner loop)
        
    It returns:
    an array of dicts with fields: TaskName, TaskID
    '''
    
    csvOut=[]
    toExtract = retrievePage(whichData, page=1)
    totalPages = toExtract['total_pages']
    totalEntries = toExtract['total_entries']
    perPage = toExtract['per_page']
    
    for pg in range(1,totalPages+1):
        jsonOut = retrievePage(whichData, page=pg)
        nrEntriesLeftOver = totalEntries - (pg-1)*perPage
        if nrEntriesLeftOver > 100:
            nrEntriesInLoop = 100
        else:
            nrEntriesInLoop = nrEntriesLeftOver
        #print(pg,nrEntriesInLoop)
        for entry in range(nrEntriesInLoop):
            temp = {}
            temp['TaskName'] = jsonOut[whichData][entry]['name']
            temp['TaskID'] = jsonOut[whichData][entry]['id']
            csvOut.append(temp)
    return csvOut

In [6]:
def retrievePageProjAssignment(UID,page=1,per_page=100):
    '''
    Retrieves the url with the headers dict using the authorization from
    https://help.getharvest.com/api-v2/authentication-api/authentication/authentication/
    
    It takes:
    UID: user ID 
    page: which page of the result is returned, if the number of entries is larger than
        per_page
    per_page: amount of entries per page, 1 <= per_page <= 100
    
    It returns:
    a json object containing the User Project Assignments associated to the user
    
    Example:
    retrievePage(1234567, page=2) # it retrieves the 2nd page of the project list
    '''
    url = 'https://api.harvestapp.com/v2/users/'+str(UID)+'/project_assignments'

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    payload = {'per_page':str(per_page),'page':str(page)}
    
    r = requests.get(url=url, headers=headers, params=payload)
    jsonResponse = r.json()
    #print(json.dumps(jsonResponse, sort_keys=True, indent=4))
    return jsonResponse

In [7]:
def getListOfProjectsAssociatedWithPerson(UID):
    '''
    It loops over the received pages in json form, it merges the results and it
    returns them. 
    It has to be used in combination with retrievePageProjAssignment(). 
    
    It takes:
    UID: the user ID (you have mainly to adapt
    which fields should be returned, in the inner loop)
        
    It returns:
    csvOut: the list of projects in which the user UID is involved
    '''
    
    csvOut=[]
    toExtract = retrievePageProjAssignment(UID, page=1)
    totalPages = toExtract['total_pages']
    totalEntries = toExtract['total_entries']
    perPage = toExtract['per_page']
    
    for pg in range(1,totalPages+1):
        jsonOut = retrievePageProjAssignment(UID, page=pg)
        nrEntriesLeftOver = totalEntries - (pg-1)*perPage
        if nrEntriesLeftOver > 100:
            nrEntriesInLoop = 100
        else:
            nrEntriesInLoop = nrEntriesLeftOver
        for entry in range(nrEntriesInLoop):
            csvOut.append(jsonOut['project_assignments'][entry]['project']['id'])
            
    return csvOut

In [8]:
def getListOf_ProjectsAndProjAssign_AssociatedWithPerson(UID):
    '''
    It loops over the received pages in json form, it merges the results and it
    returns them. 
    It has to be used in combination with retrievePageProjAssignment(). 
    
    It takes:
    UID: the user ID (you have mainly to adapt
    which fields should be returned, in the inner loop)
        
    It returns:
    csvOut: the list of projects in which the user UID is involved
    '''
    
    csvOutProj=[]
    csvOutProjAssign=[]
    toExtract = retrievePageProjAssignment(UID, page=1)
    totalPages = toExtract['total_pages']
    totalEntries = toExtract['total_entries']
    perPage = toExtract['per_page']
    
    for pg in range(1,totalPages+1):
        jsonOut = retrievePageProjAssignment(UID, page=pg)
        nrEntriesLeftOver = totalEntries - (pg-1)*perPage
        if nrEntriesLeftOver > 100:
            nrEntriesInLoop = 100
        else:
            nrEntriesInLoop = nrEntriesLeftOver
        #print(pg,nrEntriesInLoop)
        for entry in range(nrEntriesInLoop):
            #print(jsonOut['project_assignments'][entry]['project']['id'])
            csvOutProj.append(jsonOut['project_assignments'][entry]['project']['id'])
            csvOutProjAssign.append(jsonOut['project_assignments'][entry]['id'])
    return csvOutProj,csvOutProjAssign

In [9]:
def GetUserAssignment(jsonElement, UserID):
    for i in jsonElement['user_assignments']:
        temp = i['id']
        if i['user']['id']==UserID:
            break
        else:
            pass
    return temp

## Building DataFrames

It retrieves all the **tasks**, **projects** and **clients** from the Harvest server.  
It then converts the data into DataFrames

In [10]:
TasksDict = createJsonDict_Tasks()
ProjectsDict = createJsonDict_Projects()
ClientsDict = createJsonDict_Clients()

In [11]:
taskDf = pd.DataFrame(TasksDict)
projDf = pd.DataFrame(ProjectsDict)
clientDf = pd.DataFrame(ClientsDict)

In [12]:
# taskDf.to_csv('ToRemoveTasks.csv')
# clientDf.to_csv('ToRemoveClients.csv')
taskDf.shape, projDf.shape, clientDf.shape

((160, 2), (2099, 4), (197, 2))

In [1]:
taskDf.head()

In [2]:
projDf.head()

In [3]:
clientDf.head(10)

# Assign a new colleague to all projects of another colleague

In [16]:
ExistingColleaguesID = 2336517
ColleagueID = '3153729'

In [17]:
ListOfProjects = getListOfProjectsAssociatedWithPerson(ExistingColleaguesID)

In [18]:
len(ListOfProjects)

49

In [19]:
ListOfProjects[:2]

[22269620, 22269043]

In [4]:
allLog = []
for n,i in enumerate(ListOfProjects):
    print(n,i)
    url = 'https://api.harvestapp.com/v2/projects/'+str(i)+'/user_assignments'

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    
    payload = {'per_page':str(100),'page':str(1),'user_id':ColleagueID}#

    r = requests.post(url=url, headers=headers, params=payload)
    jsonResponse = r.json()
    allLog.append(jsonResponse)



# Delete all projects from the account of a colleague
To delete Projects from an account you must know not only the ProjID but also the UserAssignmentID. This has the same mechanich as for "Assign / remove to / from a colleague the Project Manager status" (read that!).  

    TL;DR  
    This means that the very same project (with the same ProjectID) has different UserAssignmentIDs for different users (UserIDs).

In [ ]:
ColleagueIDWhoseProjectsIWillDelete = 1170159

In [ ]:
ListOfProjects = getListOfProjectsAssociatedWithPerson(ColleagueIDWhoseProjectsIWillDelete)

In [ ]:
len(ListOfProjects), ListOfProjects[:2]

First collect for a certain ColleagueID the tuples `(ProjectID, UserAssignmentID)`

In [ ]:
allLog = []
Project_UserAssignment_Tuple = []
for n,i in enumerate(ListOfProjects):
    print(n,i)
    url = 'https://api.harvestapp.com/v2/projects/'+str(i)+'/user_assignments'

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    payload = {'per_page':str(100),'page':str(1)}#

    r = requests.get(url=url, headers=headers, params=payload)
    jsonResponse = r.json()
    allLog.append(jsonResponse)
    

    Project_UserAssignment_Tuple.append((i,GetUserAssignment(jsonResponse,ColleagueIDWhoseProjectsIWillDelete)))

In [ ]:
Project_UserAssignment_Tuple

In [ ]:
for a,b in Project_UserAssignment_Tuple:
    print(a,b)

And now for each pait `(ProjectID, UserAssignmentID)` we call a `delete`-command.

In [ ]:
allLog = []
for projID, projAssID in Project_UserAssignment_Tuple:
    print(projID,projAssID)
    url = 'https://api.harvestapp.com/v2/projects/'+str(projID)+'/user_assignments/'+str(projAssID)

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}

    r = requests.delete(url=url, headers=headers) #, params=payload)
    jsonResponse = r.json()
    allLog.append(jsonResponse)
    print(jsonResponse)

# Assign / remove to / from a colleague the Project Manager status

This is a bit more tricky than the rest...  
Look at the manual: https://help.getharvest.com/api-v2/projects-api/projects/user-assignments/.  

To begin with, set the `ExistingColleaguesID`, that is the person you are getting the list of projects from. _In principle it can be the same person._  
Then we can start.

* First get the list of user_assignments via `GET /v2/projects/{PROJECT_ID}/user_assignments`, taken from the paragraph "List all user assignments for a specific project".  
* You get in response the list of people involved with this project, with their UserID and their UserAssignmentID. This means that the very same project (with the same ProjectID) has different UserAssignmentIDs for different users (UserIDs).  
* In the JSON response pick the UserAssignmentID corresponding to the person you are looking for.  
* From the paragraph "Update a user assignment" you can now update the status of the User by issuing `PATCH /v2/projects/{PROJECT_ID}/user_assignments/{USER_ASSIGNMENT_ID}` with `-d '{"is_project_manager":"true"}`. Note that this is possible because the status of the UserAssignmentID you are modifying refers **only** to the person with the UserID you have just found before.  
* make a foor loop in which you search for the UserAssignmentIDs in the list of ProjectIDs and then you modify them.  

In [ ]:
ExistingColleaguesID = 2079842

In [ ]:
ListOfProjects = getListOfProjectsAssociatedWithPerson(ExistingColleaguesID)

In [ ]:
len(ListOfProjects)

This function gets the response from the server (in json form), looks for the `UserAssignmentID` corresponding to the `UserID` and returns it. 

In [ ]:
GetUserAssignment(jsonElement, UserID) # defined above

Here I collect from the list of `ProjectsIDs` corresponding to the `ExistingColleaguesID` from which I want to copy.  
I get a `Project_UserAssignment_Tuple`: each line has a `ProjectID` and the corresponding `UserAssignmentIDs`. The problem is now that I want only the `UserAssignmentID` that corresponds to the `ProjectID` (this is automatically set in the call to the server: I can ask only for one project) **and** `UserID` (the person I want to copy the privileges to). To this purpose I use the `GetUserAssignment()` function.  
Given the response from the server, it selects only the `UserAssignmentIDs` corresponding to my `UserID`.  
At the end, I have the `Project_UserAssignment_Tuple` ready for the next step

In [ ]:
allLog = []
Project_UserAssignment_Tuple = []
for n,i in enumerate(ListOfProjects):
    print(n,i)
    url = 'https://api.harvestapp.com/v2/projects/'+str(i)+'/user_assignments'

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    payload = {'per_page':str(100),'page':str(1)}#

    r = requests.get(url=url, headers=headers, params=payload)
    jsonResponse = r.json()
    allLog.append(jsonResponse)
    

    Project_UserAssignment_Tuple.append((i,GetUserAssignment(jsonResponse,2079842)))

The last step is the one in which we make the changes on the Harvest Server.  
I iterate on the `Project_UserAssignment_Tuple` and I make calls to the server for the Project and the UserAssignment I'm interested in. The dictionary `data = {'is_project_manager':'true'}` sets the privileges I want to change.

We are done!

In [ ]:
allLog2 = []
for n,i in enumerate(Project_UserAssignment_Tuple):
    print(n,i)
    url = 'https://api.harvestapp.com/v2/projects/'+str(i[0])+'/user_assignments/'+str(i[1])

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    payload = {'per_page':str(100),'page':str(1)}#
    data = {'is_project_manager':'true'}

#     r = requests.get(url=url, headers=headers, params=payload)
    r = requests.patch(url=url, headers=headers, params=data)#, params=payload)
    jsonResponse = r.json()
    allLog2.append(jsonResponse)



# Small script to assign a new colleague to existing projects

The program works as follows:  
* Select which keyword you are looking in the DataFrame of Projects+Clients;   
* Take the ProjectIDs of the Project-Client pairs that match the search 
* Iterate on the right url (https://api.harvestapp.com/v2/projects/'__XXX__'/user_assignments)

In [ ]:
projDf.loc[projDf.ProjName.str.contains('SMM - Blog'),'ProjID']

In [ ]:
allLog = []
for n,i in enumerate(projDf.loc[projDf.ProjName.str.contains('SMM - Blog'),'ProjID']):
    print(n)
    url = 'https://api.harvestapp.com/v2/projects/'+str(i)+'/user_assignments'

    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    payload = {'per_page':str(100),'page':str(1),'user_id':'2279908'}

    r = requests.post(url=url, headers=headers, params=payload)
    jsonResponse = r.json()
    allLog.append(jsonResponse)


In [ ]:
allLog

## Functions for retrieving, modifying and adding a task assignment

In [ ]:
def RetrTaskAssignment(whichProjID,whichTaskID=None):
    '''
    A simplified version of retrievePage() specialized for retrieving the task assignments for a project. 
    CAVEAT: no effort has been undertaken to take care of the case where many pages (NrOfEntries > 100)
    are returned
    
    It takes:
    whichProjID: the project ID
    whichTaskID: the assignment task ID. If None, then all the assignment tasks are retrieved
    
    It returns:
    a json object containing the retrieved data.
    
    Example:
    RetrTaskAssignment(1234567)
    '''
    if whichTaskID==None:
        Url = 'https://api.harvestapp.com/v2/projects/'+str(whichProjID)+'/task_assignments'#/'+str(whichTaskID)
    else:
        Url = 'https://api.harvestapp.com/v2/projects/'+str(whichProjID)+'/task_assignments/'+str(whichTaskID)
    Headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'Content-type': 'application/json'}
    Payload = {'task_id':str(whichTaskID),'is_active':'true','billable':'true',
               'hourly_rate':'0.0'}
    #return Url, Headers, Payload
    r = requests.get(url=Url, headers=Headers)#,params=Payload)
    jsonResponse = r.json()
    return jsonResponse

In [ ]:
def AddTaskAssignment(whichProjID,taskIDtoBeAdded):
    '''
    A simplified version of retrievePage() specialized for retrieving the task assignments for a project. 
    CAVEAT: no effort has been undertaken to take care of the case where many pages (NrOfEntries > 100)
    are returned
    
    It takes:
    whichProjID: the project ID
    whichTaskID: the assignment task ID. If None, then all the assignment tasks are retrieved
    
    It returns:
    a json object containing the retrieved data.
    
    Example:
    AddTaskAssignment(16747673,1232132)
    '''
    
    Url = 'https://api.harvestapp.com/v2/projects/'+str(whichProjID)+'/task_assignments'
    Headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'Content-type': 'application/json'}
    Payload = {'task_id':str(taskIDtoBeAdded),'is_active':'true','billable':'true',
               'hourly_rate':'0.0'}
    #return Url, Headers, Payload
    r = requests.post(url=Url, headers=Headers, params=Payload)
    jsonResponse = r.json()
    return jsonResponse

In [ ]:
def createJsonDict_TasksAssignment(whichProjID):
    '''
    It loops over the received pages in json form, it merges the results and it
    returns them. 
    It has to be used in combination with RetrTaskAssignment(). 
    
    It takes:
    whichProjID: the project ID
    
    It returns:
    an array of dicts with fields: ProjectID, TaskAssignmentID,TaskName,TaskID.
    
    NB the **commented** code can be found in createJsonDict_Projects()
    '''
    
    csvOut=[]
    toExtract = RetrTaskAssignment(whichProjID)
    totalPages = toExtract['total_pages']
    totalEntries = toExtract['total_entries']
    perPage = toExtract['per_page']
    
    for pg in range(1,totalPages+1):
        jsonOut = RetrTaskAssignment(whichProjID)
        nrEntriesLeftOver = totalEntries - (pg-1)*perPage
        if nrEntriesLeftOver > 100:
            nrEntriesInLoop = 100
        else:
            nrEntriesInLoop = nrEntriesLeftOver
        #print(pg,nrEntriesInLoop)
        for entry in range(nrEntriesInLoop):
            temp = {}
            temp['ProjectID'] = whichProjID
            temp['TaskAssignmentID'] = jsonOut['task_assignments'][entry]['id']
            temp['TaskName'] = jsonOut['task_assignments'][entry]['task']['name']
            temp['TaskID'] = jsonOut['task_assignments'][entry]['task']['id']
            csvOut.append(temp)
    return csvOut

In [ ]:
def ArchiveTaskAssignment(whichProjID,whichTaskAssignmentID):
    '''
    The function archives the task assignment with whichTaskAssignmentID from the project with whichProjID.
    In practice it sets the parameter 'is_active' to false
    
    It takes:
    whichProjID: the project ID
    whichTaskID: the assignment task ID. If None, then all the assignment tasks are retrieved
    
    It returns:
    a json object containing the response from server.
    
    Example:
    ArchiveTaskAssignment(123456,654321)
    '''
    Url = 'https://api.harvestapp.com/v2/projects/'+str(whichProjID)+'/task_assignments/'+str(whichTaskAssignmentID)
    headers = {'User-Agent': 'MyApp (my.email@company.de)', # Use your Mail Address
               'Authorization': 'Bearer 1234_mytoken', # Use your Token 
               'Harvest-Account-ID': '987654', # Use your Account-ID
               'content-type': 'application/json'}
    Payload = {'is_active':'false'}

    r = requests.patch(url=Url, headers=Headers, params=Payload)
    jsonResponse = r.json()
    return jsonResponse

In [ ]:
testDf = pd.DataFrame(createJsonDict_TasksAssignment(16799525))

In [ ]:
testDf

In [ ]:
TaskAssToErase_seoTextInt = testDf[testDf.TaskName == 'SEO Texte intern'].TaskAssignmentID.values[0]
TaskAssToErase_richContInt = testDf[testDf.TaskName == 'Rich Content intern'].TaskAssignmentID.values[0]  
TaskAssToErase_seoTextInt, TaskAssToErase_richContInt

In [ ]:
ArchiveTaskAssignment(16799525,TaskAssToErase_seoTextInt)

In [ ]:
ArchiveTaskAssignment(16799525,TaskAssToErase_richContInt)

## Adding tasks to selected Projects
In this case search all projects containing 'SEO - CONTENT' in ProjName and add to those the three tasks below:  
* 12425833	Textverlängerung
* 12425832	Textanpassung
* 12425824	Textkürzung

Some trivial aspects:
This function **connects** existing Tasks to existing Projects --> If the Task doesn't exist, you have to first create it.  
The function needs both the TaskID and the ProjectID to establish the connection.  
Please note that it uses some standard settings:  
* 'is_active':'true'  
* 'billable':'true'  
* 'hourly_rate':'0.0'  
If some of them do not fit your needs you have to change them

In [ ]:
projDf

In [ ]:
for entry in range(len(projDf)):
    if projDf.loc[entry,'ProjName']=='SEO - Content': #search the projects with 'SEO - Content' in the name
        tempProjectID = projDf.loc[entry,'ProjID'] # take the ID of the project
        print(entry,tempProjectID)
        AddTaskAssignment(tempProjectID,5011801) 
#         AddTaskAssignment(tempProjectID,12425833) #Textverlängerung
#         AddTaskAssignment(tempProjectID,12425832) #Textanpassung
#         AddTaskAssignment(tempProjectID,12425824) #Textkürzung
        